In [2]:
import numpy as np
import REMG
import time
REMG.list_devices()

,device,name,description,hwid,vid,pid,serial_number,location,manufacturer,product,interface
0,/dev/cu.BLTH,cu.BLTH,n/a,n/a,NaN,NaN,None,None,None,None,None
1,/dev/cu.Bluetooth-Incoming-Port,cu.Bluetooth-Incoming-Port,n/a,n/a,NaN,NaN,None,None,None,None,None
2,/dev/cu.BoseQC35II,cu.BoseQC35II,n/a,n/a,NaN,NaN,None,None,None,None,None
3,/dev/cu.usbserial-0001,cu.usbserial-0001,CP2102 USB to UART Bridge Controller - CP2102 ...,USB VID:PID=10C4:EA60 SER=0001 LOCATION=20-4.3,4292.0,60000.0,0001,20-4.3,Silicon Labs,CP2102 USB to UART Bridge Controller,CP2102 USB to UART Bridge Controller


In [4]:
rheo = REMG.remgSerialInterface('/dev/cu.usbserial-0001')

Opened serial port /dev/cu.usbserial-0001 at 921600 baud


In [5]:
rheo.start_reading()

Started Serial While Loop
Started reading thread


In [9]:
rheo.clear_state_buffer()
while True:
    if rheo.have_state():
        state_id, state_ts, state = rheo.get_state()
        print(state_id, state_ts, list(state[rheo.EMG_IDX]), list(state[rheo.RMG_IDX]), end=" "*50+'\r')
    else:
        time.sleep(0.005)

KeyboardInterrupt: 

# Teleplot


In [10]:
import socket
teleplotAddr = ("127.0.0.1",47269)
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

def sendTelemetry(name, value):
    now = time.time() * 1000
    msg = name+":"+str(now)+":"+str(value)+"|g"
    sock.sendto(msg.encode(), teleplotAddr)

def sendManyTelemetry(s: str):
    sock.sendto(s.encode(), teleplotAddr)
EMG_LABELS= ["EMG%i"%i for i in range(len(rheo.EMG_IDX))]
RMG_LABELS = ["RMG%i"%i for i in range(len(rheo.RMG_IDX))]


rheo.clear_state_buffer()
while True:
    if rheo.have_state():
        state_id, state_ts, state = rheo.get_state()
        print(state_id, state_ts, list(state[rheo.EMG_IDX]), list(state[rheo.RMG_IDX]), end=" "*50+'\r')
        sendManyTelemetry('\n'.join(list(map(lambda x: f'{x[0]}:{x[1]}', zip(EMG_LABELS, state[rheo.EMG_IDX])))))
        sendManyTelemetry('\n'.join(list(map(lambda x: f'{x[0]}:{x[1]}', zip(RMG_LABELS, state[rheo.RMG_IDX])))))
    else:
        time.sleep(0.005)


KeyboardInterrupt: 

Exception in thread Thread-6 (read_serial):
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/opto3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3/envs/opto3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mike/motorica/Data/RheoMG/rheo/REMG/remgSerialInterface.py", line 66, in read_serial
    data_raw = self.ser.read(self.data_len)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/envs/opto3/lib/python3.11/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)
